In [ ]:
# Header starts here.
from sympy.physics.units import *
from sympy import *

# Rounding:
import decimal
from decimal import Decimal as DX
from copy import deepcopy
def iso_round(obj, pv, rounding=decimal.ROUND_HALF_EVEN):
    import sympy
    """
    Rounding acc. to DIN EN ISO 80000-1:2013-08
    place value = Rundestellenwert
    """
    assert pv in set([
        # place value   #  round to:
        1,              #  1
        0.1,            #  1st digit after decimal
        0.01,           #  2nd
        0.001,          #  3rd
        0.0001,         #  4th
        0.00001,        #  5th
        0.000001,       #  6th
        0.0000001,      #  7th
        0.00000001,     #  8th
        0.000000001,    #  9th
        0.0000000001,   # 10th
        ])
    objc = deepcopy(obj)
    try:
        tmp = DX(str(float(objc)))
        objc = tmp.quantize(DX(str(pv)), rounding=rounding)
    except:
        for i in range(len(objc)):
            tmp = DX(str(float(objc[i])))
            objc[i] = tmp.quantize(DX(str(pv)), rounding=rounding)
    return objc

# LateX:
kwargs = {}
kwargs["mat_str"] = "bmatrix"
kwargs["mat_delim"] = ""

# kwargs["symbol_names"] = {FB: "F^{\mathsf B}", }

# Units:
(k, M, G ) = ( 10**3, 10**6, 10**9 )
(mm, cm) = ( m/1000, m/100 )
Newton = kg*m/s**2
Pa     = Newton/m**2
MPa    = M*Pa
GPa    = G*Pa
kN     = k*Newton
deg    = pi/180

half = S(1)/2

# Header ends here.
#
x, a, q, EI, l = var("x, a, q, EI, l")

x = var("x")

w = (x*l - x*x)
W = a*w
Wp = diff(W,x)
Wpp = diff(Wp, x)

f1 = Wpp + q/2/EI*w
f2 = w

# Integrate:
I = integrate(f1*f2, x)

# Definite integral:
Idef = I.subs(x,l)

pprint("\nGalerkin Equation:")
eq = Eq(Idef,0)
pprint(eq.simplify())

sol=solve(eq,a)
asol = sol[0]
pprint("\nSolution for Parameter a:")
pprint(asol)

# Substitute to get
Wsol = W.subs(a,asol)
pprint("\nSolution for w:")
pprint(Wsol)

# W(l/2)
pprint("\nw(l/2):")
Wl2=Wsol.subs(x,l/2)
pprint(Wl2)

pprint("\nPrinciple of Zero Virtual Power (1):")
a = var("a")
W = a*(l*x - x*x)

Wp = diff(W, x)
Wpp = diff(Wp, x)
pprint("\nw:")
pprint(W)

M = -EI*Wpp
Ui = integrate( -M*Wpp, (x,0,l) ) / 2
Ua = integrate( q*W, (x,0,l) )
pprint("\nInternal: Ui / EI:")
pprint((Ui/EI).simplify())
pprint("\nExternal: Ua:")
pprint(Ua.simplify())

eq = Eq(diff(Ui-Ua, a))
sol = solve(eq, a)
asol = sol[0]
pprint("\nSolution for Parameter a:")
pprint(asol)
pprint("\nw / ( (ql²) / (24 EI) ):")
tmp = W.subs([(a, asol), ])
tmp /= (q*l**2) / (24*EI)
tmp = tmp.simplify()
pprint(tmp)

pprint("\nPrinciple of Zero Virtual Power (2):")
a4 = var("a4")
W = a4*( l**3*x - 2*l*x**3 + x**4 )
Wp = diff(W, x)
Wpp = diff(Wp, x)
pprint("\nw:")
pprint(W)

M = -EI*Wpp
Ui = integrate( -M*Wpp, (x,0,l) ) / 2
Ua = integrate( q*W, (x,0,l) )
pprint("\nInternal: Ui / EI:")
pprint((Ui/EI).simplify())
pprint("\nExternal: Ua:")
pprint(Ua.simplify())

pprint("\nSolution:")
eq = Eq(diff(Ui-Ua, a4))
sol = solve([eq], [a4])
pprint(sol)
a4s = sol[a4]

pprint("\nw / ( (ql²) / (24 EI) ):")
Ws = W.subs([ (a4, a4s), ])
fac = (q*l**2) / (24*EI)
tmp = Ws
tmp /= fac
tmp = tmp.simplify()
pprint(tmp)

pprint("\nw(l/2):")
tmp = Ws
tmp = tmp.subs(x,l/2)
pprint(tmp)
